# SoulScript Agent Walkthrough (LLM, Context, Memory)

Runnable guide focused on the agent stack (LLM calls, context aggregation, and memory). Scheduler/tick orchestration is deliberately omitted. The seeds here are hardcoded to match the repo format so you can run this notebook anywhere in the repo.

## Repo + env setup
Purpose: locate the repo root, add it to `sys.path`, and load `.env` so LLM keys are available.

Example input: current working directory.
Example output: resolved `REPO_ROOT` and `sys.path` confirmation.

In [1]:
from pathlib import Path
import os, sys

def _find_repo_root() -> Path:
    candidates = [Path.cwd(), *Path.cwd().parents]
    if "__file__" in globals():
        here = Path(__file__).resolve()
        candidates.append(here.parent)
        candidates.extend(here.parents)
    for candidate in candidates:
        if (candidate / "soulscript" / "core" / "config.py").exists():
            return candidate
    return Path.cwd()

REPO_ROOT = _find_repo_root()
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

env_path = REPO_ROOT / ".env"
if env_path.exists():
    for line in env_path.read_text(encoding="utf-8").splitlines():
        stripped = line.strip()
        if not stripped or stripped.startswith("#") or "=" not in stripped:
            continue
        key, value = stripped.split("=", 1)
        if key and value and key not in os.environ:
            os.environ[key] = value

print("repo_root:", REPO_ROOT)
print("sys.path contains repo_root:", str(REPO_ROOT) in sys.path)


repo_root: c:\Users\Computia.me\Documents\Github\Project-SoulScript
sys.path contains repo_root: True


## Architecture 
- **Seeds → NPC containers:** authored JSON → `NPCTruth` → `NPCProfile` (immutable truth), wrapped by `NPC` for mood/self-perception.
- **LLM clients:** `OllamaLLMClient` (local) and `OpenRouterLLMClient` (remote) share the same prompt schema and validation.
- **Policy + context aggregation:** `apply_policy` picks allowed actions + tool outputs; `_build_llm_context` merges profile, memory, world context, and tools for the prompt.
- **Validator + decision:** `_validated_decision` enforces action constraints and retries/falls back on junk outputs.
- **Memory:** `ConversationMemory` stores short-term lines and long-term summaries; feeds back into context. Relationship graph is available but the scheduler loop is not used here.

## Hardcoded seeds


Example input: two NPC truth dicts.
Example output: `NPCProfile` objects ready for the agent stack.

In [2]:
from soulscript.core.types import NPCProfile, NPCTruth, TraitVector

seed_a = {
    "id": "npc_mara",
    "name": "Mara",
    "age": 29,
    "race": "half-orc",
    "sex": "female",
    "sexual_orientation": "bisexual",
    "backstory": "Hired blade looking for steady guard work and a warm seat.",
    "traits": "Tall, scarred, soft-voiced, quick to check others are okay.",
    "motivation": "Find steady guard work near the fire.",
    "role": "visitor",
    "trait_inputs": {"honesty": 60, "patience": 45, "optimism": 10, "charisma": 20},
    "traits_truth": {
        "kindness": 55, "bravery": 75, "extraversion": -5, "ego": -10, "honesty": 60,
        "curiosity": 25, "patience": 50, "optimism": 5, "intelligence": 50, "charisma": 15,
    },
    "traits_self_perception": {
        "kindness": 60, "bravery": 80, "extraversion": 0, "ego": -5, "honesty": 65,
        "curiosity": 20, "patience": 45, "optimism": 10, "intelligence": 45, "charisma": 20,
    },
    "inventory": ["patched cloak", "armband", "memento ring"],
    "schedule": {"morning": "market", "afternoon": "tavern_common", "evening": "tavern_common"},
    "initial_relationships": {},
}
seed_b = {
    "id": "npc_riven",
    "name": "Riven",
    "age": 31,
    "race": "human",
    "sex": "male",
    "sexual_orientation": "heterosexual",
    "backstory": "Courier who keeps stopping in for news and warm meals.",
    "traits": "Lean, weathered, easy smile, always checking doorways.",
    "motivation": "Trade gossip for meals and rest.",
    "role": "local",
    "trait_inputs": {"honesty": 55, "patience": 30, "optimism": 25, "charisma": 35},
    "traits_truth": {
        "kindness": 40, "bravery": 55, "extraversion": 25, "ego": 5, "honesty": 55,
        "curiosity": 45, "patience": 35, "optimism": 30, "intelligence": 45, "charisma": 35,
    },
    "traits_self_perception": {
        "kindness": 45, "bravery": 60, "extraversion": 30, "ego": 10, "honesty": 50,
        "curiosity": 50, "patience": 30, "optimism": 35, "intelligence": 40, "charisma": 40,
    },
    "inventory": ["mail satchel", "folded maps"],
    "schedule": {"morning": "tavern_common", "afternoon": "market", "evening": "tavern_common"},
    "initial_relationships": {},
}

profiles = [NPCProfile(truth=NPCTruth(**seed_a)), NPCProfile(truth=NPCTruth(**seed_b))]
print([p.truth.npc_id for p in profiles])


['npc_mara', 'npc_riven']


## LLM calls: local Ollama and OpenRouter option
Purpose: exercise both LLM providers with the same minimal context. After this, default to OpenRouter for the rest of the notebook.

Example input: shared context + allowed actions.
Example output: `Decision` objects with action/line.

In [5]:
from soulscript.core import config
from soulscript.core.llm import OllamaLLMClient, OpenRouterLLMClient
from soulscript.core.types import Action, ActionType

sample_profile = profiles[0]
partner_id = profiles[1].truth.npc_id
trait_scale = {
    "kindness": "-100=cruel, 100=kind",
    "bravery": "-100=cowardly, 100=brave",
    "extraversion": "-100=withdrawn, 100=gregarious",
    "ego": "-100=selfless, 100=proud",
    "honesty": "-100=dishonest, 100=honest",
    "curiosity": "-100=apathetic, 100=seeking",
    "patience": "-100=impulsive, 100=patient",
    "optimism": "-100=cynical, 100=hopeful",
    "intelligence": "-100=dim, 100=brilliant",
    "charisma": "-100=off-putting, 100=magnetic",
}
shared_context = {
    "profile": {
        "name": sample_profile.truth.name,
        "backstory": sample_profile.truth.backstory,
        "motivation": sample_profile.truth.motivation,
        "traits_summary": sample_profile.truth.traits,
        "trait_scale": trait_scale,
    },
    "state": {
        "mood": 55,
        "traits_truth": sample_profile.truth.traits_truth.as_dict(),
    },
    "short_term": [f"{partner_id}: The stew smells great."],
    "recent_thread": [f"{partner_id}: The stew smells great."],
    "last_partner_line": "The stew smells great.",
    "long_term": "",
    "global_facts": list(config.GLOBAL_KNOWLEDGE),
    "conversation_partner": partner_id,
    "interaction_tone": "Quick hello in the tavern.",
}
allowed_actions = [
    Action(action_type=ActionType.SPEAK, target_id=partner_id),
    Action(action_type=ActionType.IDLE, target_id=partner_id),
]

# orig_provider = config.LLM_PROVIDER
# config.LLM_PROVIDER = "ollama"
# local_llm = OllamaLLMClient()
# local_decision = local_llm.select_action(sample_profile.truth.npc_id, shared_context, allowed_actions)
config.LLM_PROVIDER = "openrouter"
remote_llm = OpenRouterLLMClient()
remote_decision = remote_llm.select_action(sample_profile.truth.npc_id, shared_context, allowed_actions)
config.LLM_PROVIDER = orig_provider

#print("Local/Ollama:", local_decision)
print("OpenRouter:", remote_decision)


OpenRouter: npc_id='npc_mara' selected_action=Action(action_type=<ActionType.SPEAK: 'speak'>, target_id='npc_riven', metadata={}) reason='Engaging positively about the stew to create a friendly atmosphere.' dialogue_line='It really does! I hope it tastes as good as it smells.' confidence=0.6


## Context aggregation (policy + prompt bundle)
Purpose: show how `apply_policy` and `_build_llm_context` produce the LLM-ready payload.

Example input: NPC profile, mood, world context for a focused pair.
Example output: allowed actions, tool outputs, and the merged context dict.

In [6]:
from soulscript.core.runtime import apply_policy, _build_llm_context, ConversationMemory, RelationshipGraph

memory = ConversationMemory()
relationships = RelationshipGraph()
relationships.bootstrap([p.truth.npc_id for p in profiles], seeds={})

world_context = {
    "focus_target": partner_id,
    "defer_effects": True,
    "allowed_npcs": [sample_profile.truth.npc_id, partner_id],
}
allowed_actions_ctx, tool_outputs = apply_policy(sample_profile, "speak", world_context)
llm_context = _build_llm_context(
    sample_profile,
    npc_mood=55,
    conversation_memory=memory,
    context=world_context,
    tool_outputs=tool_outputs,
    relationships=relationships,
)
print("Allowed actions:", [a.action_type.value for a in allowed_actions_ctx])
print("Tool outputs:", tool_outputs)
print("Context keys:", sorted(llm_context.keys()))
print("Short term:", llm_context.get("short_term"))
print("Global facts:", llm_context.get("global_facts"))


Allowed actions: ['speak', 'idle']
Tool outputs: {'inventory': ['patched cloak', 'armband', 'memento ring'], 'schedule': {'morning': 'market', 'afternoon': 'tavern_common', 'evening': 'tavern_common'}}
Context keys: ['allowed_npcs', 'conversation_partner', 'defer_effects', 'focus_target', 'global_facts', 'interaction_tone', 'inventory', 'last_partner_line', 'long_term', 'profile', 'recent_thread', 'schedule', 'short_term', 'state']
Short term: []
Global facts: ['You are in a cozy fantasy tavern.', 'Half the people here are locals and half are visitors.', 'Keep dialogue short and friendly.']


## Memory section (short-term + summary)
Purpose: demonstrate `ConversationMemory.record` and `context_bundle` that feeds context aggregation.

Example input: a few dialogue lines between Mara and Riven.
Example output: ordered short-term lines and auto-generated long-term summary when the trigger is hit.

In [7]:
from datetime import datetime

memory = ConversationMemory()
now = datetime.utcnow()
memory.record("npc_mara", "npc_riven", "npc_mara", "Quiet night by the fire.", now)
memory.record("npc_mara", "npc_riven", "npc_riven", "Warm food helps the road.", now)
memory.record("npc_mara", "npc_riven", "npc_mara", "You look tired from travel.", now)
memory.record("npc_mara", "npc_riven", "npc_riven", "Long ride, but worth the company.", now)

bundle = memory.context_bundle("npc_mara", "npc_riven", existing_summary=None)
print("Short term:", bundle["short_term"])
print("Long term summary:", bundle["long_term"])
print("Global facts:", bundle["global_facts"])


Short term: ['npc_riven: Long ride, but worth the company.', 'npc_mara: You look tired from travel.', 'npc_riven: Warm food helps the road.', 'npc_mara: Quiet night by the fire.']
Long term summary: I caught up with npc_riven. They mentioned 'Warm food helps the road.' I replied 'Quiet night by the fire.'
Global facts: ['You are in a cozy fantasy tavern.', 'Half the people here are locals and half are visitors.', 'Keep dialogue short and friendly.']


## Relationship perception 
Purpose: show the perception edge after the conversation snippets; no scheduler needed.

Example input: adjust relation directly.
Example output: trust/affinity and perceived traits.

In [8]:
edge = relationships.adjust_relation(
    source_id="npc_mara",
    target_id="npc_riven",
    trust_delta=2,
    affinity_delta=2,
    trait_deltas={"kindness": 1},
    summary=bundle["long_term"],
    timestamp=datetime.utcnow(),
)
print({"trust": edge.trust, "affinity": edge.affinity, "summary": edge.summary, "traits": edge.traits.as_dict()})


{'trust': 2, 'affinity': 2, 'summary': "I caught up with npc_riven. They mentioned 'Warm food helps the road.' I replied 'Quiet night by the fire.'", 'traits': {'kindness': 1, 'bravery': 0, 'extraversion': 0, 'ego': 0, 'honesty': 0, 'curiosity': 0, 'patience': 0, 'optimism': 0, 'intelligence': 0, 'charisma': 0}}
